# Google Earth Engine dataset import and querying

In [ ]:
import requests
import json
# Use your authorization key
api_token = input()

# Do not commit your key!

In [50]:
# Declare the API base url
api_url = "https://api.apihighways.org"
# Convenience function for registering datasets
def register(payload, endpoint = 'v1/dataset', api_url = api_url, api_token = api_token):
    print("Doing authenticated request to the API")
    headers = {'User-Agent': 'python-requests', 'Content-Type': 'application/json', 'Authorization': f'Bearer {api_token}'}
    result = requests.post('/'.join((api_url, endpoint)), json = payload, headers = headers)
    status_code, result = result.status_code, result.text
    print(f"Status code: {status_code}")
    return json.loads(result)

Google Earth Engine supports several kinds of datasets. What kind of queries are possible is determined by its kind. First, we'll register an [Image](https://developers.google.com/earth-engine/image_overview). The IDs can be obtained from the [Google Earth Engine explorer](https://explorer.earthengine.google.com/#workspace) --linking directly to the dataset is not possible.

In [51]:
umd_hansen = register(
    payload = {
        # These are the required fields
        "name": "umd_hansen_global_forest_change_2015", # The name will determine the slug
        "connectorType": "rest",
        "provider": "gee", # Google Earth Engine are of provider type 'gee'
        "application": ["data4sdgs"],
        "tableName": "UMD/hansen/global_forest_change_2015_v1_3" # GEE image ID
})
umd_hansen

Doing authenticated request to the API
Status code: 200


{'data': {'attributes': {'application': ['data4sdgs'],
   'attributesPath': None,
   'blockchain': {},
   'clonedHost': {},
   'connectorType': 'rest',
   'connectorUrl': None,
   'dataPath': None,
   'env': 'production',
   'errorMessage': None,
   'geoInfo': False,
   'layerRelevantProps': [],
   'legend': {'country': [], 'date': [], 'nested': [], 'region': []},
   'mainDateField': None,
   'name': 'umd_hansen_global_forest_change_2015',
   'overwrite': False,
   'protected': False,
   'provider': 'gee',
   'published': True,
   'slug': 'umd_hansen_global_forest_change_2015',
   'status': 'pending',
   'subtitle': None,
   'tableName': 'UMD/hansen/global_forest_change_2015_v1_3',
   'taskId': None,
   'type': None,
   'updatedAt': '2018-05-14T17:09:54.102Z',
   'userId': '599c0e5458113b0001d75c47',
   'verified': False,
   'widgetRelevantProps': []},
  'id': '4f507b39-2653-41f9-a880-766baa5a0c47',
  'type': 'dataset'}}

In [52]:
umd_hansen["data"]["id"]

'4f507b39-2653-41f9-a880-766baa5a0c47'

Some examples of possible queries to `Image `s

In [63]:
def query(dataset_id, query):
    return json.loads(
        requests.get(
            f"{api_url}/query/{dataset_id}?sql={query}"
        ).text
    )

query(
    umd_hansen["data"]["id"],
    "SELECT ST_HISTOGRAM(raster, lossyear, 15, true) FROM 'UMD/hansen/global_forest_change_2015_v1_3'"
)

{'data': [{'st_histogram': {'lossyear': [[0.0, 7081461.411764656],
     [1.0666666666666667, 0.0],
     [2.1333333333333333, 0.0],
     [3.2, 0.0],
     [4.266666666666667, 1.0],
     [5.333333333333333, 0.0],
     [6.4, 0.0],
     [7.466666666666667, 1.0],
     [8.533333333333333, 1.0],
     [9.6, 1.0],
     [10.666666666666666, 2.0],
     [11.733333333333333, 0.0],
     [12.8, 0.0],
     [13.866666666666667, 4.0],
     [14.933333333333334, 2.0]]}}],
 'meta': {}}

Queries can be also made to the generic 'query' endpoint. The slug must be provided.

In [64]:
def query_no_id(query):
    return json.loads(
        requests.get(
            f"{api_url}/query?sql={query}"
        ).text
    )

dataset_slug = umd_hansen["data"]["attributes"]["slug"]
query_no_id(
    f"SELECT ST_HISTOGRAM(raster, lossyear, 15, true) FROM {dataset_slug}"
)

{'data': [{'st_histogram': {'lossyear': [[0, 7081461.411764656],
     [1.0666666666666667, 0],
     [2.1333333333333333, 0],
     [3.2, 0],
     [4.266666666666667, 1],
     [5.333333333333333, 0],
     [6.4, 0],
     [7.466666666666667, 1],
     [8.533333333333333, 1],
     [9.6, 1],
     [10.666666666666666, 2],
     [11.733333333333333, 0],
     [12.8, 0],
     [13.866666666666667, 4],
     [14.933333333333334, 2]]}}],
 'meta': {}}

[ImageCollections](https://developers.google.com/earth-engine/ic_creating) are sets of images. They can be registered as datasets, but queries are done to the information of the ImageCollection, not the Images themselves. We'll use the Copernicus S2 1C imagery as an example.

In [65]:
# Registering the dataset
copernicus = register(
    payload = {
        # These are the required fields
        "name": "COPERNICUS S2 Level 1C imagery", # The name will determine the slug
        "connectorType": "rest",
        "provider": "gee", # Google Earth Engine are of provider type 'gee'
        "application": ["data4sdgs"],
        "tableName": "COPERNICUS/S2" # GEE image ID
})
copernicus

Doing authenticated request to the API
Status code: 200


{'data': {'attributes': {'application': ['data4sdgs'],
   'attributesPath': None,
   'blockchain': {},
   'clonedHost': {},
   'connectorType': 'rest',
   'connectorUrl': None,
   'dataPath': None,
   'env': 'production',
   'errorMessage': None,
   'geoInfo': False,
   'layerRelevantProps': [],
   'legend': {'country': [], 'date': [], 'nested': [], 'region': []},
   'mainDateField': None,
   'name': 'COPERNICUS S2 Level 1C imagery',
   'overwrite': False,
   'protected': False,
   'provider': 'gee',
   'published': True,
   'slug': 'COPERNICUS-S2-Level-1C-imagery',
   'status': 'pending',
   'subtitle': None,
   'tableName': 'COPERNICUS/S2',
   'taskId': None,
   'type': None,
   'updatedAt': '2018-05-14T17:35:33.537Z',
   'userId': '599c0e5458113b0001d75c47',
   'verified': False,
   'widgetRelevantProps': []},
  'id': 'c38673bf-29a6-4b75-9099-4314ce269a31',
  'type': 'dataset'}}

In [67]:
# The output will be metadata relating to the Images in the ImageCollection
query(
    copernicus["data"]["id"],
    "SELECT * from 'COPERNICUS/S2' limit 10"
)

{'data': [{'CLOUDY_PIXEL_PERCENTAGE': 0.0,
   'CLOUD_COVERAGE_ASSESSMENT': 0.010707692307692307,
   'DATASTRIP_ID': 'S2A_OPER_MSI_L1C_DS_EPA__20160606T223605_S20150627T102531_N02.02',
   'DATATAKE_IDENTIFIER': 'GS2A_20150627T102537_000062_N02.02',
   'DATATAKE_TYPE': 'INS-NOBS',
   'DEGRADED_MSI_DATA_PERCENTAGE': 0.0,
   'ECMWF_DATA_REF': 'S2__OPER_AUX_ECMWFD_FAKE_19800101T000000_V19800101T000000_19800101T000000',
   'FORMAT_CORRECTNESS_FLAG': 'PASSED',
   'GENERAL_QUALITY_FLAG': 'PASSED',
   'GENERATION_TIME': 1465314766000,
   'GEOMETRIC_QUALITY_FLAG': 'PASSED',
   'GRANULE_ID': 'S2A_OPER_MSI_L1C_TL_EPA__20160606T223605_A000062_T31RCL_N02.02',
   'GRI_FILENAME': 'S2A_OPER_AUX_GRI065_PDMC_20130621T120000_S20130101T000000',
   'IERS_BULLETIN_FILENAME': 'S2__OPER_AUX_UT1UTC_PDMC_20150625T000000_V20150626T000000_20160625T000000',
   'MEAN_INCIDENCE_AZIMUTH_ANGLE_B1': 103.965471634376,
   'MEAN_INCIDENCE_AZIMUTH_ANGLE_B10': 102.245777794173,
   'MEAN_INCIDENCE_AZIMUTH_ANGLE_B11': 103.0914

In [83]:
# Other example
gridmet = register(
    payload = {
        # These are the required fields
        "name": "GRIDMET: University of Idaho Gridded Surface Meteorological Dataset", # The name will determine the slug
        "connectorType": "rest",
        "provider": "gee", # Google Earth Engine are of provider type 'gee'
        "application": ["data4sdgs"],
        "tableName": "IDAHO_EPSCOR/GRIDMET" # GEE image ID
})
gridmet

Doing authenticated request to the API
Status code: 200


{'data': {'attributes': {'application': ['data4sdgs'],
   'attributesPath': None,
   'blockchain': {},
   'clonedHost': {},
   'connectorType': 'rest',
   'connectorUrl': None,
   'dataPath': None,
   'env': 'production',
   'errorMessage': None,
   'geoInfo': False,
   'layerRelevantProps': [],
   'legend': {'country': [], 'date': [], 'nested': [], 'region': []},
   'mainDateField': None,
   'name': 'GRIDMET: University of Idaho Gridded Surface Meteorological Dataset',
   'overwrite': False,
   'protected': False,
   'provider': 'gee',
   'published': True,
   'slug': 'GRIDMET-University-of-Idaho-Gridded-Surface-Meteorological-Dataset_1',
   'status': 'pending',
   'subtitle': None,
   'tableName': 'IDAHO_EPSCOR/GRIDMET',
   'taskId': None,
   'type': None,
   'updatedAt': '2018-05-14T17:58:41.604Z',
   'userId': '599c0e5458113b0001d75c47',
   'verified': False,
   'widgetRelevantProps': []},
  'id': '53f86a5d-dc34-4cab-9738-0ccc88eec636',
  'type': 'dataset'}}

In [84]:
query(
    gridmet["data"]["id"],
    "select * from 'IDAHO_EPSCOR/GRIDMET' limit 10"
)

{'data': [{'status': 'permanent',
   'system:asset_size': 27576721,
   'system:footprint': {'coordinates': [[-108.5453124696084,
      49.46257585003468],
     [-113.9593748919571, 49.46257584884527],
     [-124.85170940475892, 49.4620585601637],
     [-124.83351210930364, 24.93736504761038],
     [-116.66640625473028, 24.963260502976674],
     [-109.44765621625152, 24.96326051909946],
     [-102.22890624654843, 24.963260484807783],
     [-96.81484372072272, 24.995926577231895],
     [-92.30312493334293, 24.982315870677233],
     [-87.79140622110124, 24.995926539340573],
     [-83.27968748970726, 24.98231580541047],
     [-77.86562494351487, 24.982315792477387],
     [-71.54921870653465, 24.96326053409072],
     [-66.99145134929572, 25.003807969975153],
     [-66.97329042095092, 49.4620585688528],
     [-81.47499992614492, 49.4625758671273],
     [-90.49843747394075, 49.462575896553865],
     [-97.71718744154877, 49.462575845088494],
     [-108.5453124696084, 49.46257585003468]],
    '

[FeatureCollections](https://developers.google.com/earth-engine/feature_collections) are the homolog of vector layers in the Google Earth Engine ecosystem.

In [85]:
glims_2016 = register(
    payload = {
        # These are the required fields
        "name": "Global Land Ice Measurements 2016", # The name will determine the slug
        "connectorType": "rest",
        "provider": "gee", # Google Earth Engine are of provider type 'gee'
        "application": ["data4sdgs"],
        "tableName": "GLIMS/2016" # GEE image ID
})

sea_temp_salinity = register(
    payload = {
        # These are the required fields
        "name": "HYCOM + NCODA Global 1/12° Analysis", # The name will determine the slug
        "connectorType": "rest",
        "provider": "gee", # Google Earth Engine are of provider type 'gee'
        "application": ["data4sdgs"],
        "tableName": "HYCOM/GLBu0_08/sea_temp_salinity" # GEE image ID
})

(glims_2016, sea_temp_salinity)

Doing authenticated request to the API
Status code: 200
Doing authenticated request to the API
Status code: 200


({'data': {'attributes': {'application': ['data4sdgs'],
    'attributesPath': None,
    'blockchain': {},
    'clonedHost': {},
    'connectorType': 'rest',
    'connectorUrl': None,
    'dataPath': None,
    'env': 'production',
    'errorMessage': None,
    'geoInfo': False,
    'layerRelevantProps': [],
    'legend': {'country': [], 'date': [], 'nested': [], 'region': []},
    'mainDateField': None,
    'name': 'Global Land Ice Measurements 2016',
    'overwrite': False,
    'protected': False,
    'provider': 'gee',
    'published': True,
    'slug': 'Global-Land-Ice-Measurements-2016',
    'status': 'pending',
    'subtitle': None,
    'tableName': 'GLIMS/2016',
    'taskId': None,
    'type': None,
    'updatedAt': '2018-05-14T18:05:29.282Z',
    'userId': '599c0e5458113b0001d75c47',
    'verified': False,
    'widgetRelevantProps': []},
   'id': '7416e29b-908a-4e06-a9b0-8bebb1e435b7',
   'type': 'dataset'}},
 {'data': {'attributes': {'application': ['data4sdgs'],
    'attributes

In [86]:
query(
    glims_2016["data"]["id"],
    "SELECT * from 'GLIMS/2016' where area > 0.09 limit 10"
)

{'data': [{'analysts': 'Rabatel, Antoine',
   'anlys_id': 121404,
   'anlys_time': '2013-03-20T00:00:00',
   'area': 0.0905,
   'chief_affl': "Laboratoire de Glaciologie et G�ophysique de l'Environnement (LGGE)",
   'db_area': 0.090459,
   'geog_area': 'French Alps',
   'glac_id': 'G006237E44775N',
   'glac_name': 'Plat',
   'glac_stat': 'exists',
   'length': 506.485,
   'line_type': 'glac_bound',
   'local_id': 'None',
   'max_elev': 2970,
   'mean_elev': 2892,
   'min_elev': 2802,
   'parent_id': '',
   'primeclass': 0,
   'proc_desc': 'Automatic delineation using combination of SWIR, NIR, and GREEN; individual checking and manual corrections when necessary; First, NDSI was calculated on the images.  From the resulting maps, glacier outlines were extracted.  Second, the shapefiles result',
   'rc_id': 33,
   'rec_status': 'okay',
   'release_dt': '2013-12-25T14:01:59',
   'src_date': '1988-09-12T00:00:00',
   'subm_id': 569,
   'submitters': 'Rabatel, Antoine',
   'system:index': '0

In [88]:
query(
    sea_temp_salinity["data"]["id"],
    "SELECT * from 'HYCOM/GLBu0_08/sea_temp_salinity' limit 10"
)

{'data': [{'experiment': '190',
   'system:asset_size': 743581341,
   'system:footprint': {'coordinates': [[-180.0, -90.0],
     [180.0, -90.0],
     [180.0, 90.0],
     [-180.0, 90.0],
     [-180.0, -90.0]],
    'type': 'LinearRing'},
   'system:index': '1992100200',
   'system:time_end': 717984000000.0,
   'system:time_start': 717984000000.0},
  {'experiment': '190',
   'system:asset_size': 744232323,
   'system:footprint': {'coordinates': [[-180.0, -90.0],
     [180.0, -90.0],
     [180.0, 90.0],
     [-180.0, 90.0],
     [-180.0, -90.0]],
    'type': 'LinearRing'},
   'system:index': '1992100300',
   'system:time_end': 718070400000.0,
   'system:time_start': 718070400000.0},
  {'experiment': '190',
   'system:asset_size': 745010814,
   'system:footprint': {'coordinates': [[-180.0, -90.0],
     [180.0, -90.0],
     [180.0, 90.0],
     [-180.0, 90.0],
     [-180.0, -90.0]],
    'type': 'LinearRing'},
   'system:index': '1992100400',
   'system:time_end': 718156800000.0,
   'system:ti